In [1]:
from IPython.core.display import HTML
HTML('''<style>
        .container { width:100% !important; }
        </style>
     ''')

# A Backtracking CSP-Solver

The module `extractVariables` implements the function $\texttt{extractVars}(e)$ that takes a *Python* expression $e$ as its argument and that returns the set of all variables occurring in $e$.

In [2]:
import extractVariables as ev

In [3]:
ev.extractVars('1.0 * x + y*z**2')

{'x', 'y', 'z'}

In [4]:
def arb(M):
    "Return some element from the set M."
    for x in M:
        return x

Backtracking is simulated by raising the `Backtrack` exception.  We define this new class of exceptions so that we can distinguish `Backtrack` exceptions from ordinary exceptions.  This is done by creating a new, empty class that is derived from the class `Exception`.  The details of defining new classes are deferred to the lecture on *algorithms* in the second semester.

In [5]:
class Backtrack(Exception):
    pass

The function $\texttt{solve}(\texttt{CSP})$ tries to compute a solution for the *constraint satisfaction problem* $\texttt{CSP}$.  A *constraint satisfaction problem* is defined as a triple of the form
$$ \langle\texttt{Variables}, \texttt{Values}, \texttt{Constraints}\rangle $$
where
<ol>
    <li>$\texttt{Variables}$ is a list of the variables occurring in the CSP,</li>
    <li>$\texttt{Values}$ is the set of values that can be substitued for the variables, and</li>
    <li>$\texttt{Constraints}$ is the set of formulas that need to be satisfied.</li>
</ol>

In [6]:
def solve(CSP):
    "Compute a solution for the given constraint satisfaction problem."
    Variables, Values, Constraints = CSP
    CSP = (Variables, Values, [(f, ev.extractVars(f) & set(Variables)) for f in Constraints])
    try:
        return backtrack_search({}, CSP)
    except Backtrack:
        return  # no solution found

In [ ]:
def backtrack_search(Assignment, CSP):
    """
    Given a partial variable assignment, this function tries to complete this assignment
    towards a solution of the CSP.
    """
    print(Assignment)
    (Variables, Values, Constraints) = CSP
    if len(Assignment) == len(Variables):
        return Assignment
    var = [x for x in Variables if x not in Assignment][0] # pick a variable
    for value in Values:
        try:
            if isConsistent(var, value, Assignment, Constraints):
                NewAssign      = Assignment.copy()
                NewAssign[var] = value
                return backtrack_search(NewAssign, CSP)
        except Backtrack:
            continue
    # all values have been tried without success, no solution has been found
    raise Backtrack()

The function $\texttt{isConsistent}(\texttt{var}, \texttt{value}, \texttt{Assignment}, \texttt{CSP})$ takes for arguments:
<ol>
    <li>$\texttt{var}$ is a variable that does not occur in $\texttt{Assignment}$,</li>
    <li>$\texttt{value}$ is a value that can be substituted for this variable,</li>
    <li>$\texttt{Assignment}$ is a partial variable assignment, and </li>
    <li>$\texttt{CSP}$ is a constraint satisfaction problem.</li>
</ol>
This function returns <tt>True</tt> iff the partial variable assignment 
$$\texttt{Assignment} \cup \{\langle\texttt{var} \mapsto\texttt{value}\rangle\}$$
is consistent with all the constraints occurring in $\texttt{CSP}$.

In [ ]:
def isConsistent(var, value, Assignment, Constraints):
    NewAssign      = Assignment.copy()
    NewAssign[var] = value
    return all(eval(f, NewAssign) for (f, Vs) in Constraints
                                  if var in Vs and Vs <= NewAssign.keys()
              )

The function $\texttt{queensCSP}$ returns a CSP that codes the 8-Queens-Problem.

## The 8-Queens-Problem

The function $\texttt{queensCSP}$ returns a CSP that codes the 8-Queens-Problem.  For $i\in\{1,\cdots,8\}$, the variable $\texttt{Q}i$ specifies the column of the queen that is placed in row $i$.

In [ ]:
def queensCSP():
    "Returns a CSP coding the 8 queens problem."
    S            = range(1, 8+1)          # used as indices
    Variables    = [ f'Q{i}' for i in S ]
    Values       = { 1, 2, 3, 4, 5, 6, 7, 8 }
    SameColumn   = { f'Q{i} != Q{j}' for i in S for j in S if i < j }
    SameDiagonal = { f'abs(Q{i}-Q{j})!={j-i}' for i in S for j in S if i < j }
    return (Variables, Values, SameColumn | SameDiagonal | {'Q1 != 1'})

In [ ]:
queensProblem = queensCSP()

In [ ]:
queensProblem

In [ ]:
%time Solution = solve(queensProblem)

In [ ]:
print(Solution)

In [ ]:
def printSolution(Assignment):
    if Assignment == None:
        print('no solution found')
        return
    n      = len(Assignment)
    Queens = [0] * (n+1)
    for row in range(1, n+1):
        Queens[row] = Assignment[f'Q{row}']
    print((4 * n + 1) * "-")
    for row in range(1, n+1):
        line = "|"
        for col in range(1, n+1):
            if Queens[row] == col:
                line += " Q |"
            else:
                line += "   |"
        print(line)
        print((4*n+1) * "-")

In [ ]:
printSolution(Solution)